# 클로저 : 프로그래밍의 즐거움 Chapter 6. 지연과 불변성

## Younggun Na (amazingguni@gmail.com)

## Overview

- 불변성(immutability)
- 영속성 구조 설계하기(트리)
- 키워드의 용도(e.g., :keyword)
- 지연 퀵 정렬

## 6.1 불변성(immutability)

Programming language 차원에서 `불변성`를 추구하면 어려운 문제점들을 단순하게 해결할 수 있다.

#### 예정된 불변

모든 불변성 객체가 가질 수 있는 속성들은 **생성 시점에 정의되고 그 이후에는 변경될 수 없음**

> `예정론`: 전적으로 자유로운 것이란 존재하지 않고 모든 것은 계획되어 있다는 개념 -> 즉 한 번 만들어지면 뭘해도 바뀌지 않는다는 말...

#### 합의에 의한 불변

시스템 내에서 불변성이 전반적으로 잘 활용되려면 불변성에 대한 전체적인 약속이 필요

가령 *Java*에서 불변 클래스를 생성하려면

1. 클래스 자체와 그에 해당하는 모든 필드가 final로 선언되어야 함
2. 객체를 생성하는 과정에서 this를 통한 객체의 참조가 불가능함
3. 내부의 불변 객체들은 클래스 안에서 복사만 허용되고 외부로 참조가 되지 않아야 한다.

```java
final class A{
    final private int num;
    final private object obj;
    public A(num){
        this.num = num;
    }
    
    public object getObject(){
        return object.clone();
    }
}
```

클로저는 핵심적인 데이터 구조들을 통해 언어의 특징으로써 불변성을 직접 지원

- 불변성 구조를 만드는 것과 구현할 내용 각각에 의해 발생하는 복잡성을 분리
- 구현할 내용에 더욱 집중할 수 있다

### 6.1.2 불변성은 왜 사용하는가?

클로저의 불변성 데이터 구조는 단순히 언어 자체에 국한되는 것이 아니라, 언어의 핵심적인 철학에까지 연결되어 있다.

#### 불변(invariant)

불변성을 기반으로 하는 프로그래밍에서는 클래스나 함수에 대해 인스턴스가 특정 상태에 있을 때 값을 입력하려 하면 에러가 발생하도록 한다.

가변성 시스템에 불변성을 부여하려면 오직 생성에서만 값을 입력할 수 있도록 하여야 함

#### 객체 상태 예측

불변 객체의 운명은 이미 정해져 있기 때문에 예측이 쉽다.

#### 동일성의 의미

가변성의 세계에서는 동일성이 의미가 없음

- 두 객체가 현재 시점에서 동일하다고 해도 이후까지는 보장이 되지 않는다.
- 만일 두 객체가 동일하지 않다면,, 그들은 같다고 말할 수 없다.

``` java
class A{
    public int num;
    
    public A(int num){
        this.num = num;
    }
}
A a = new A(3); // 이 때는 a.num은 3이다.
a.num = 5; // 하지만.. a.num이 5로 변했다.


foo(a); // 인수로 전달되는 a의 속성은 전달 당시의 값과 같다고 보장되지 않는다.
```

불변성 객체는 지금 당장 동일하다면, 항상 동일할 것이라고 보장한다.

#### 쉬운 공유

어떤 객체가 절대 변경되지 않으리라는 확신이 있다면 **참조**를 제공함으로써 간편하게 공유할 수 있다.

* 만약 java에서 그렇게 하려면 방어적인 복사가 필요하다 (*Object.clone같은.*)

#### 간접성 감소

클로저에는 `가변성 참조`만 있다.

![title](6_var.png)

* Java에서는 기본적으로 가변성 데이터를 가리키는 참조가 있다.
* 이로 불필요한 복잡성이 크게 감소



#### 병행 프로그래밍을 뒷받침하는 불변성

불변성 객체는 항상 스레드에 안전하다.

* 변경으로 인한 에러에 구애받지 않고 다른 스레드에 자유롭게 전달할 수 있음

## 6.2 구조적 공유: 영속적 구조

리스트는 가장 단순한 형태의 *공유 구조* 타입

> 공유 구조에서는 변경되는 값을 위한 메모리만 새로 할당하고 변하지 않는 나머지 부분은 원래 객체에서 공유

In [5]:
(def l `(1 2 3))
(def m1 (rest l))
(def m2 (into `(4 5 6) l))
(def r (reverse m1))

(println l)
(println m1)
(println m2)
(println r)

(1 2 3)
(2 3)
(3 2 1 4 5 6)
(3 2)


#'user/l #'user/m1 #'user/m2 #'user/r nil nil nil nil

![title](6_clojure_list.gif)

In [14]:
(def mapv1 {:name "paul" :age 45})
(def mapv2 (assoc mapv1 :sec :male))

(println mapv1)
(println mapv2)

{:name paul, :age 45}
{:name paul, :age 45, :sec :male}


#'user/mapv1 #'user/mapv2 nil nil

항상 복사를 하면 효율적이지 않기 때문에 클로저의 컬렉션에서는 공유 구조를 가지고 있다.

In [3]:
(def baselist (list :barnabas :adam))
(def list1 (cons :willie baselist))
(def list2 (cons :phoenix baselist))

(println list1)
(println list2)

(:willie :barnabas :adam)
(:phoenix :barnabas :adam)


#'user/baselist #'user/list1 #'user/list2 nil nil

In [4]:
(= (next list1) (next list2))
(identical? (next list1) (next list2))

true true

#### 불변 객체로 이루어진 트리

여기서 만들어볼 트리는 Left Right Value를 가진 간단한 트리이다.

맵에 넣어보면 아래와 같다

In [8]:
{:val 5, :L nil, :R nil}

{:val 5, :L nil, :R nil}

In [ ]:
값으로 5를 가지고 왼쪽 가지가 비어있고 오른쪽 가지도 비어있는 하나의 노드

트리에 노드를 추가해주는 함수를 만들 수 있다.

In [6]:
(defn xconj [tree value]
  (cond
    (nil? tree) {:val value, :L nil, :R nil}))

#'user/xconj

In [7]:
(xconj nil 5)

{:val 5, :L nil, :R nil}

이제 첫 노드를 추가할 차례이다.

tree가 nil이 아니고 자신보다 value가 작은 경우에는 왼쪽 가지에 추가한다.

그를 위해 아래와 같은 비교가 필요해진다.

```clojure
(< value (:value tree))
```

또한, 아래와 같이 기존 노드의 값을 변경하지 않고 새로운 노드를 만들어서 복사해야 한다.

```clojure
(:value (:value tree),
  :L (여기에 새로운 노드를 입력)
  :R (:R tree))
```

#### 전체 코드

In [16]:
(defn xconj [tree value]
  (cond
    (nil? tree)           {:val value, :L nil, :R nil}
    (< value (:val tree)) {:val (:val tree),
                           :L   (xconj (:L tree) value)
                           :R   (:R tree)}
    :else                 {:val (:val tree),
                           :L   (:L tree)
                           :R   (xconj (:R tree) value)}
  )
)

#'user/xconj

In [17]:
(def tree1 (xconj nil 5))
(println tree1)
(def tree1 (xconj tree1 3))
(println tree1)
(def tree1 (xconj tree1 2))
(println tree1)

{:val 5, :L nil, :R nil}
{:val 5, :L {:val 3, :L nil, :R nil}, :R nil}
{:val 5, :L {:val 3, :L {:val 2, :L nil, :R nil}, :R nil}, :R nil}


#'user/tree1 nil #'user/tree1 nil #'user/tree1 nil

In [18]:
tree1

{:val 5, :L {:val 3, :L {:val 2, :L nil, :R nil}, :R nil}, :R nil}

데이터가 잘 들어갔는지 확인하기 위해 순서대로 조회하면서 출력해주는 함수를 만들어보자

In [19]:
(defn xseq [tree]
  (when tree
    (concat (xseq (:L tree)) [(:val tree)] (xseq (:R tree)))))

#'user/xseq

In [20]:
(xseq tree1)

(2 3 5)

In [21]:
(def tree2 (xconj tree1 7))
(xseq tree2)

#'user/tree2 (2 3 5 7)

![title](6_tree.jpg)

In [22]:
(identical? (:L tree1) (:L tree2))

true

이 예제는 모든 클로저의 영속적 컬렉션들이 공통적으로 갖는 몇가지 특징들을 보여준다.

1. 모든 *변경*은 일단 새로운 루트 노드를 생성하고, 새 값이 삽입될 위치의 트리 경로에 새 노드를 추가한다.
2. 값과 변경되지 않는 가지가 복사되는 것이 아니라, 기존의 노드에서 새 노드로 그 참조가 복사된다.
3. 스레드에 안전하다. 
  - *xconf*를 호출하기 전에 존재하던 객체들은 어떤 방법으로든 변경되지 않기 때문
  
하지만 위의 코드는 좋지 않은 품질의 코드다.

1. 이진 트리 방식
2. 숫자 외에 다른 값을 저장할 수 없다.
3. 트리의 깊이가 깊어지면 오버 플로그 발생 가능성이 있다.
4. *xseq*가 지연 시퀀스가 아닌 트리 전체를 복사하는 방식이다.
5. 비 균형적인 트리를 생성할 수 있다.

클로저는 메모리 공간 절약을 위해 지연 시퀀스라는 개념에 크게 의존하고 있는데,, 이건 다음에..

## 6.3 지연